## 用户画像的构建
### 用户画像里面有什么？什么样的内容能存到用户画像？
- **用户基本信息**

    一般在用户注册时，就能获取到用户的信息
    - 性别：
        - 1. 注册时填写 
        - 2. 算法预测，根据用户的点击行为预测
    - 年龄：
        - 1. 注册填写身份信息，手机号，可间接获取用户年龄  
        - 2. 算法预测，根据用户的点击行为预测
    - 地理位置：IP地址，GPS定位，常驻地地址，实时GPS定位
    - 机型：手机机型，操作系统
    - 时间：第一次进入时间，最后一次退出时间，用户活跃时长，累计浏览时长，累计播放时长等
    
    
- **用户行为信息**

    根据用户的点击曝光，收集用户行为信息，包括用户点击的分类，关键词，Tag，主题，短期，中期，长期画像中存储的，也就是和内容画像相关的一些信息
    
    用户的画像可以根据用户行为，按照时间切片进行分别存储，比如，用户看某些分类，只在短期内感兴趣，过一段时间之后就不感兴趣了。这种份额里，就能存在短期画像
    
    - 短期画像：3-10小时
    - 中期画像：一周或两周
    - 长期画像：10周或半年
    
    **短期画像案例**：
    我最近刷头条，看到河豚很可爱，并且我还对进行了搜索，在之后的推荐中，也大量给我推荐了相关的内容，但是我也只是在那几个小时内刷完，之后就不会再感兴趣，再之后，给我推相关的内容，我也很少点，因为短期的大量曝光已经满足了用户对这类内容的需求，用户对这类内容没有长期感兴趣，就会随着时间推移，从画像中淘汰

    **中期画像案例**：
    比如现在有个场景，比如现在正是高考结束的时候，网上很多资讯，文章都是关于高考的，对于高考这么一个主题来说，用户不会一直对这个领域感兴趣，可能就是在高考这段时间感兴趣。那这个就可以认定是一个中期画像，因为基于时间切片，有一个时间滑动窗口，当在最近一周时间内，用户浏览过的关于高考相关的文章的分类，关键词，tag，主题进入用户的中期画像，如果用户只在最近一周关注，那我们可以基于中期画像召回，满足用户需求，当用户慢慢关注较少，随着时间的推移，这类画像中关于高考的信息，也就被慢慢的从画像中淘汰出去。

    **长期画像案例**：
    比如，我是一个美食领域爱好者，对美食内容很感兴趣，基本上只要更新新内容，推给我，只要内容不反感，我都会点，那这类兴趣就属于长期兴趣，因为我持续半年都在点相关的内容。美食类的分类，主题，就可以从长期画像中稳定召回。


### 画像构建
实际业务中，用户画像中，比如上面讲到的，短期，中期，长期画像分别存储，对用户的行为数据，根据时间进行切片，比如，短期画像的存储，最近3-10小时(不等)的用户行为数据，将用户感兴趣的分类，关键词，主题存到画像中，每个分类下的曝光e，点击c，ctr分数，根据用户对每个tag的点击，评论，收藏，分享等行为，进行加权，计算一个权值，存入画像中，以供后续模型建模或者用户画像召回

## 建立倒排索引
### 文章倒排索引使用场景
每个分类下，有若干文章列表，对每个文章进行打分，之后将文章按照分值进行倒排存储

当你召回某个分类下的文章时，就直接从这个分类下的文章列表取数据，一般都是取TopN，这次取了数据之后，之前取的N条就被过滤了，下一次又是取下面的TopN。

### 如何对内容进行打分？
- **全冷用户的场景**

    在产品属于冷启动状态，往往没有积累到用户行为，我们可以通过运营手段，加上规则和策略，整理热门内容进行推荐。人为控制给用户推荐，达到推荐可控，可解释，少badcase。
    
    - 运营梳理热门分类，并对热门分类的某些内容设定相应的权重，比如，娱乐，科技，财经领域是大众都爱看的内容，就能将这类内容头部的，认为用户爱看的内容，直接推给用户，并制定一些相关规则，控制用户的频次，以及推荐的条数，为算法积累画像。
    - 对优质文章根据标题，文本长度，评论数，收藏数，阅读数等一系列的信息，进行加权，计算一个相对合理的 base score
        - 这类信息如何得知：一般很多内容如果是爬虫爬过来的，这些内容在其他网站就有用户行为信息，曝光，点击，分分享，收藏，点赞数，都有，可以基于这些进行加权

    
- **有用户行为场景**

    当客户端已经不再是冷启动阶段，也会借助算法进行推荐的时候，此时的文章倒排，就要借助 base score，以及用户行为数据，进行加权，计算一个得分，这个得分，反应了端内大部分用户的兴趣偏好，从而，对于一个新来的用户来说，经过大量的用户验证过的内容，再推荐给用户，至少用户不会反感。
    
    - 文章内容层面的分数：文章标题权重，内容是否优质，内容长短，作者等信息，加权计算的分数
    - 用户层面的行为分数：
        - 用户的CTR：CTR可以进行平滑，当点击和曝光次数太少的时候，CTR往往很高，不够置信，比如，曝光5条，点击2条，此时CTR已经40%，这时CTR其实很高，但是能说明用户对这类内容一定很感兴趣吗？不一定，当推荐足够多的时候，CTR才有置信度。我们可以对点击和曝光进行平滑，比如威尔逊置信CTR。
        - 用户的点赞，评论，收藏，评论
        - 用户的负反馈：点击x，标明不感兴趣，反感等，评论是负面情感
        - 用户的阅读时长，平均阅读时长
        - 时间衰减：某一篇文章在最近一段时间很火，通过计算分数，它排名很靠前，但是这个事件已经过去了，用户已经不把焦点放在这个上面了，就要根据时间衰减
        
### hotboost分数
hotboost分数，和上面说的文章倒排分数是一个意思，就相当于是一个通过所有用户对文章的行为，进行加权，计算的一个文章倒排的分数

当对新用户进行推荐的时候，就可以基于这个分数进行倒排推荐，这个分数都是被其它用户验证过的，相对来说，有一定的价值，当这个用户行为足够多，就可以进行ctr排序，就可以根据这个用户对每个分类下的文章，建立倒排，进行个性化排序

### 倒排索引分数计算公式伪代码

In [1]:
import numpy as np


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def calc_hotboost_score(pv, ev, alpha, beta, shareNum, commentNum, storeNum, upvoteNum, durationScore, readRateScore):
    """
    :param pv:                  文章点击量
    :param ev:                  文章曝光量
    :param alpha:               ctr平滑超参
    :param beta:                ctr平滑超参
    :param ctrScore:            ctr得分
    :param actScore:            文章行为得分,分享，评论，收藏，点赞
    :param readScore:           文章阅读时长得分,平均阅读时长
    :param negativeScore:       文章负反馈得分,负面评论
    :return:
    """
    ctrScore = (pv + alpha) / (ev + alpha + beta)
    actScore = np.log(1 + (pv + shareNum + commentNum + storeNum + upvoteNum))
    readScore = (durationScore + readRateScore) / 2
    negativeScore = 0 # 有负反馈，就根据负反馈的行为，对这个进行一个加权，如果没有负反馈，这个值就是0
    totalScore = ctrScore + actScore + readScore + negativeScore
    newhotboost = sigmoid(totalScore)
    return newhotboost

## 基于用户画像的推荐
### 构建用户画像
- **数据**
    - 文本数据，10000 条数据，10个分类 ['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
    - 对文本抽取关键词，并进行存储
    - 基于用户的行为，进行画像构建
    
        因为我们目前还没有用户行为数据，所以本次的项目，用户行为数据缺失，目前为了让项目流程能正常运行，采用的是随机生成用户感兴趣的分类和关键词数据，并存到用户画像
        
        - 10000 条文本随机生成感兴趣的分类和关键词，每个分类或关键词的数量就是差不多 2-4 个，这个可以根据实际情况调整
        - 用户画像：长，中，短，每个下面依次有感兴趣的分类和关键词(2-4)，关键词是在感兴趣的分类下的关键词
        
        1. 用户感兴趣的分类，随机选2-4个分类，并随机生成分类的权重，进行倒排
        2. 用户感兴趣的关键词，从用户感兴趣的分类下，随机选2-4个感兴趣的关键词，随机生成一个权重，进行倒排
        3. 分别对短期，中期，长期画像中分别进行类似的操作

### 建立文章倒排
- 分类下的文章倒排
    - 分类下的文章，对每个文章进行随机初始化一个权重，进行倒排
- 关键词下的文章倒排
    - 关键词下的文章，根据关键词的tfidf值建立倒排

### 基于用户的短期，中期，长期画像推荐
- 基于用户的中期，短期，长期画像，分别从画像中抽取用户感兴趣的分类，关键词下面的文章，进行推荐
- 短期，中期，长期画像召回条数可以通过模型训练进行预测，也可以直接通过规则指定

## flask部署服务
Flask是一个python web微框架，我们可以用flask简单的部署一个基于HTTP请求的服务。

Flask等web框架内部已经有了一个 WSGI server 用来接受请求，只是因为其自带的server在处理并发等情况时不够优秀。

所以在生产部署时，才需要额外的uWSGI服务器(实现WSGI server)来代替框架内部的WSGI server。

WSGI是一套接口规范。一个WSGI程序用以接受客户端请求，传递给应用，再返回服务器的响应给客户端。

下图显示了python中客户端、服务器、WSGI、应用之间的关系：

![image.png](imgs/HTTP请求.png)

从下往上开始介绍：

- 客户端：浏览器或者app。

- web服务器：Web服务器是指驻留于因特网上某种类型计算机的程序。当Web浏览器（客户端）连到服务器上并发起请求件时，服务器将处理该请求并将结果发送到该浏览器上。服务器使用HTTP（超文本传输协议）进行信息交流。

    严格意义上Web服务器只负责处理HTTP协议，用于处理静态页面的内容。而动态内容需要通过WSGI接口交给应用服务器去处理。

    Web服务器包括Nginx，Apache，IIS等。而优秀的web服务器在接收http请求时，还可以做负载均衡和反向代理等工作。

- WSGI容器：常见的WSGI容器有Gunicorn，uWSGI等。

    WSGI(Web Server Gateway Interface)是WEB服务器和web框架或web应用之间建立的一种简单通用的接口规范。有了wsgi这份接口规范，在web开发的过程中，能更加自由的选择服务器端和框架；
    
    在服务器端和框架的开发过程能够分离开来，不用过多的考虑双方具体的实现，使得服务器端和框架开发者能够专心自己领域的开发工作。而WSGI容器就是根据这套接口写出来的。
    
    WSGI容器的作用就是根据web服务器传递而来的参数构建一个让WSGI应用成功执行的环境，例如request，而且还得把WSGI应用上处理好的结果返回给web服务器。

- web框架：作用主要是方便我们开发 web 应用程序，HTTP请求的动态数据就是由 web 框架层来提供的。常见的 web 框架有Flask，Django，Tornado等。